In [ ]:
import openai
from openai import OpenAI

In [ ]:
# client = OpenAI()
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role":"user",
#                "content":"is it too late to join the course Stock Market Analytics Zoomcamp?"}]
# )

# r = response.choices[0].message.content

# from pprint import pprint
# pprint(r)

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [ ]:
from elasticsearch import Elasticsearch


In [ ]:
es_client = Elasticsearch('http://localhost:9200') 


In [ ]:
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# Delete the index if it already exists
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Deleted existing index: {index_name}")

es_client.indices.create(index=index_name, body=index_settings, )

In [ ]:
from tqdm import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
def elastic_search(query, index_name = "course-questions"):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        to_append_dict = hit['_source']
        to_append_dict['score'] = hit['_score']
        result_docs.append(to_append_dict)
        # result_docs.append(hit['_source'])
        # result_docs.append(hit['_score'])
        
    
    return result_docs

In [ ]:
elastic_search(query=query)

# Q4

In [ ]:
ml_zoomcamp_index = "ml-zoomcamp-questions"

# Delete the index if it already exists
if es_client.indices.exists(index=ml_zoomcamp_index):
    es_client.indices.delete(index=ml_zoomcamp_index)
    print(f"Deleted existing index: {ml_zoomcamp_index}")

es_client.indices.create(index=ml_zoomcamp_index, body=index_settings, )

In [ ]:
documents[0:5]

In [ ]:
for doc in tqdm(documents):
    if doc['course'] == 'machine-learning-zoomcamp':
        es_client.index(index=ml_zoomcamp_index, document=doc)

In [ ]:
elastic_search(query=query, index_name= ml_zoomcamp_index)